## Civil Aviation Authority of the Philippines - Civil Aviation Accidents, Incidents, and Serious Incidents

Data Source:
https://www.caap.gov.ph/

The *same data* can be found at [Open Data Philippines](https://data.gov.ph/?q=dataset/civil-aviation-authority-philippines-aircraft-accidents), but I chose to scrape the Civil Aviation Authority of the Philippines (CAAP) website since it contains more complete information on aircraft accidents, incidents, and serious incidents. I also found that there are some error in the data uploaded by CAAP in the Open Data Philippines website.

#### Errors found in the data uploaded at [Open Data Philippines](https://data.gov.ph/?q=dataset/civil-aviation-authority-philippines-aircraft-accidents):  
- [(Typo?) Error found at 6th row of place of occurrence column:](https://data.gov.ph/?q=dataset/civil-aviation-authority-philippines-aircraft-accidents/resource/29c1d129-11b2-4aac-89e7#{view-grid:{columnsWidth:[{column:!place_of_occurance,width:504}]}})
The place of occurrence written is 'Runway Excursion during Landing' which is not a place. The correct row is found at the [CAAP 2014 Accidents Page](https://www.caap.gov.ph/?page_id=3096).
- Occurrence is mispelled as 'occurance' on the Place of Occurrence and Type of Occurrence columns.


In [ ]:
import glob
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# Get website source
website = (requests.get("https://www.caap.gov.ph/").text) 
soup = BeautifulSoup(website, 'lxml')

In [ ]:
# Find the 'Aircraft Accident and Incident Report'
dd = soup.find("dd", {"class": "level1 nextend-nav-6425 parent"})

In [ ]:
# Get the description terms of dt('Aircraft Accident and Incident Report')
dt = dd.find_all("dt")
print(dt)

In [ ]:
accident_urls = []
incident_urls = []
serious_incident_urls = []
switch = 'a'
for item in dt:
    if item.find("a").get_text() == 'Accidents':
        continue
    if item.find("a").get_text() == 'Incidents':
        switch = 'i'
        continue
    if item.find("a").get_text() == 'Serious Incidents':
        switch = 's'
        continue
    if switch == 'a':
        accident_urls.append(item.find("a")['href'])
    elif switch == 'i':
        incident_urls.append(item.find("a")['href'])
    else:
        #print(item.find("a"))
        serious_incident_urls.append(item.find("a")['href'])

In [ ]:
# Check contents of url
print("List of accident url: ", accident_urls)
print("List of incident url: ", incident_urls)
print("List of serious incident url: ", serious_incident_urls)

In [ ]:
def get_data(url_list):
    """
    Get the concatenated DataFrame from the url in the url_list
    
    @param url_list: List of urls from the CAAP website (accident, incident, or serious_incident)
    """
    data_df = pd.DataFrame(columns=['date', 'aircraft_registration', 'aircraft_type', 'type_of_occurance',
                                           'place_of_occurance', 'status', 'report', 'report_link'])
    for url in url_list:
        print(url)
        website = (requests.get(url).text)
        soup = BeautifulSoup(website, 'lxml')
        accident_table = soup.find('tbody')
        listOfRows = accident_table.find_all('tr')
        temp_data_df = pd.DataFrame(columns=['date', 'aircraft_registration', 'aircraft_type', 'type_of_occurance',
                                           'place_of_occurance', 'status', 'report', 'report_link'])
        i = 0
        for row in listOfRows:
            listOfCells = row.find_all('td')
            if len(listOfCells) == 7 and listOfCells[6].get_text() == "Back":
                continue
            if len(listOfCells) == 7 and i > 1:
                getText = lambda x: x.get_text()
                # print(listOfCells[3])
                temp_data_df.loc[i] = list(map(getText, listOfCells)) + list([listOfCells[6].find('a')['href']])
            i += 1
        data_df = pd.concat([data_df,temp_data_df], axis = 0)
    return data_df

In [ ]:
accident_df = get_data(accident_urls)
incident_df = get_data(incident_urls)
serious_incident_df = get_data(serious_incident_urls)

In [ ]:
accident_df

In [ ]:
accident_df = accident_df.reset_index(drop=True)
incident_df = incident_df.reset_index(drop=True)
serious_incident_df = serious_incident_df.reset_index(drop=True)

#### Get geolocation of the Place of Occurrence column

Originally, I used the Nominatim's geocoders to get the location's coordinates (`from geopy.geocoders import Nominatim`); however, it returned a lot of Nones because of the inconsistent addresses. I switched to the Googlemaps api which returns the closest address string location if the exact location isn't found. For example, in the 2008 ACCIDENTS Dataset, "Taliban, Bohol" is one of the places of occurrence. Nominatim's geocoder returns None, while Google maps returns details of "Talibon, Bohol".

In [ ]:
import googlemaps
from datetime import datetime

with open('api_key.txt') as f:
    api_key = f.readline()
    f.close
gmaps = googlemaps.Client(api_key)

In [ ]:
# Geocode helper functions

# returns the geocode (json containing details of the location)
def google_get_geocode(location):
    try: 
        geocode_result = gmaps.geocode(location)
        return geocode_result[0]
    except: return None

# returns latitude given the geocode json
def google_get_latitude(geocode):
    try: 
        return geocode['geometry']['location']['lat']
    except: 
        return None

# returns longitude given the geocode json
def google_get_longitude(geocode):
    try: 
        return geocode['geometry']['location']['lng']
    except: 
        return None

# returns address given the geocode json
def google_get_address(geocode):
    try: 
        return geocode['formatted_address']
    except: 
        return None

In [ ]:
geocode_data = []
data_list = [accident_df, incident_df, serious_incident_df]

# get geocode, latitude, longitude, and formatted address of each place of occurrence 
# in the accident_df, incident_df, and serious_incident_df tables
for table in data_list:
    table['geocode'] = table.place_of_occurance.apply(google_get_geocode)
    table['latitude'] = table.geocode.apply(google_get_latitude)
    table['longitude'] = table.geocode.apply(google_get_longitude)
    table['formatted_address'] = table.geocode.apply(google_get_address)

In [ ]:
# Check how many 'None' geocode of the places of occurrence did the google maps return
print(f"Accident_df: \n{accident_df.geocode.isna().value_counts()}\n")
print(f"Incident_df: \n{incident_df.geocode.isna().value_counts()}\n", )
print(f"Serious_incident_df: \n{serious_incident_df.geocode.isna().value_counts()}", )

Manually input the geocode, latitude, longitude, and formatted_address of the 4 places of occurrence with 'None' geocodes.

In [ ]:
accident_df.loc[accident_df.geocode.isna()]

In [ ]:
temp = pd.DataFrame({'latitude' : [3.65, 14.49], 'longitude': [160.266667, 121.001500]})
accident_df.loc[accident_df.geocode.isna(), ['latitude', 'longitude']] = temp.values

- September 9, 2018
    - International Water near the Federated States of Micronesia
        - As listed from the report_link the helicopter's final position was noted at coordinates 3°39'00.0"N 160°16'00.0"E (latitude: 3.650000, longitude: 160.266667)
- May 8, 2013
    - MIA, Runway 06 (Ninoy Aquino International Airport Runway 06)
        - 14°29'54.6"N 121°00'05.4"E (latitude: 14.498500, longitude: 121.001500)

In [ ]:
incident_df.loc[incident_df.geocode.isna()]

In [ ]:
temp = pd.DataFrame({'latitude' : [14.522667, 121.005000], 'longitude': [14.524806, 121.001028]})
incident_df.loc[incident_df.geocode.isna(), ['latitude', 'longitude']] = temp.values

- Mar 12, 2010
    - Runway 13 Extension
        - NAIA Runway 13 Extension 14°31'21.6"N 121°00'18.0"E (latitude: 14.522667, longitude: 121.005000)
- Jan 24, 2010
    - Gen. Aviation Area, Domestic Airport
        - Domestic Airport, Brgy 191, Pasay City 14°31'29.3"N 121°00'03.7"E (latitude: 14.524806, longitude: 121.001028)

In [ ]:
serious_incident_df.loc[serious_incident_df.geocode.isna()]

Looks like geocode is returned successfully for all the places of occurrence in the serious incident table.

#### Clean date column

In [ ]:
from datetime import datetime

def try_parsing_date(text):
    for fmt in ('%B %d, %Y', '%b %d, %Y', '%b %d,%Y', '%B %d %Y'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    raise ValueError('no valid date format found')

accident_df['date'] = accident_df['date'].apply(try_parsing_date)
incident_df.loc[incident_df['date'].str.contains('Sept '), 'date'] = incident_df.loc[incident_df['date'].str.contains('Sept '), 'date'].str.replace("Sept", "September")
incident_df.loc[incident_df['date'].str.contains('Apirl'), 'date'] = incident_df.loc[incident_df['date'].str.contains('Apirl '), 'date'].str.replace("Apirl", "April")
incident_df['date'] = incident_df['date'].apply(try_parsing_date)
serious_incident_df['date'] = serious_incident_df['date'].apply(try_parsing_date)

#### Clean aircraft_registration column

In [ ]:
# accident_df = pd.read_csv("accidents.csv")
# incident_df = pd.read_csv("incidents.csv")
# serious_incident_df = pd.read_csv("serious_incidents.csv")

Check if there's any inconsistencies with the format of the aircraft_registration column

In [ ]:
accident_df.aircraft_registration.unique()

In [ ]:
tempRow = accident_df.loc[accident_df.aircraft_registration == "RP-R4169 & RP-R3029"]
tempRow

In [ ]:
x = tempRow.aircraft_registration.str.split(' & ').tolist()
y = tempRow.aircraft_type.str.split(' & ').tolist()

b  = pd.DataFrame([list(map(lambda pair: {'reg': pair[0], 'type': pair[1]}, zip(x[0], y[0])))], 
             index=[tempRow.date, 
                    tempRow.type_of_occurance,
                    tempRow.place_of_occurance,
                    tempRow.status,
                    tempRow.report,
                    tempRow.report_link,
                    tempRow.geocode,
                    tempRow.latitude,
                    tempRow.longitude,
                    tempRow.formatted_address]).stack()

b = b.reset_index()[[0, 
                     'date', 
                     'type_of_occurance', 
                     'place_of_occurance',
                     'status', 
                     'report', 
                     'report_link', 
                     'geocode',
                     'latitude',
                     'longitude', 
                     'formatted_address']]

b.columns = ['temp',
             'date', 
             'type_of_occurance',  
             'place_of_occurance', 
             'status', 
             'report', 
             'report_link', 
             'geocode', 
             'longitude', 
             'latitude', 
             'formatted_address']

b['aircraft_registration'] = b.temp.loc[:].apply(lambda x: x.get('reg'))
b['aircraft_type'] = b.temp.loc[:].apply(lambda x: x.get('type'))
b = b.drop('temp', axis=1)
b


In [ ]:
# drop row with inconsistent string format for aircraft type and reg
accident_df = accident_df.drop(accident_df.loc[accident_df.aircraft_registration == "RP-R4169 & RP-R3029"].index, axis=0)

In [ ]:
accident_df = pd.concat([accident_df, b], sort=True)
accident_df.reset_index(drop=True)

In [ ]:
print(incident_df.aircraft_registration.unique())
print(serious_incident_df.aircraft_registration.unique())

---
### Save to csv

In [1]:
import os
directory = 'data'
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
dict_df = {'accidents.csv': accident_df, 
           'incidents.csv': incident_df, 
           'serious_incidents.csv': serious_incident_df}
path = os.path.join(directory, "report.csv")
for key,val in dict_df.items():
    val.to_csv(os.path.join(directory, "report.csv"), encoding='utf-8', index=False)

Recommendations:

Explore other database like:
- https://aviation-safety.net/database/country/country.php?id=RP
- http://planecrashinfo.com/

In [ ]:
# accident_df.loc[90:110]